# Парсинг данных

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import re
import random
import json

import requests
from bs4 import BeautifulSoup
import base64
import lxml

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import pandas as pd

In [3]:
pd.set_option('max_colwidth', 300)
pd.set_option('display.width', 480)

In [4]:
columns_name=[
    'photo', 'date_time', 'apartment', 
    'district', 'address', 'floor_maxfloors', 
    'total_area', 'living_area', 'kitchen_area', 
    'note', 'price', 'phone_name', 
    'agency', 'email'
    ]

df = pd.DataFrame(columns=columns_name)

In [5]:
for i in range(5):
    print(f'Обрабатывается страница {i+1}')
    url = "http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=" + str(i+1)
    response = requests.get(url)
    html_content = response.content
    # взяли страницу
    soup = BeautifulSoup(html_content, "html.parser")

    # нашли таблицу
    table = soup.find("table", class_="tbrd") 


    # переберем и извлечем данные
    data = []
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        if len(columns) > 0:
            photo_url = None
            other_data = [column.text.strip() for column in columns[1:]]
            data.append([photo_url] + other_data)

    df1 = pd.DataFrame(data[3:], columns=columns_name)
    df = pd.concat([df,df1],ignore_index=True)

Обрабатывается страница 1
Обрабатывается страница 2
Обрабатывается страница 3
Обрабатывается страница 4
Обрабатывается страница 5


In [6]:
df = df.drop_duplicates().reset_index(drop=True)

In [7]:
df

,photo,date_time,apartment,district,address,floor_maxfloors,total_area,living_area,kitchen_area,note,price,phone_name,agency,email
0,None,25.0808:04,Двухкомнатная,,Галиуллина 11/2,5/5,44.00,27.00,6.00,"id:26090. \r\nВ продаже солнечная 2х комнатная квартира, расположенная на пятом этаже. Планировка квартиры «брежневка-вагон», на две стороны. Состояние квартиры простое. Окна, трубы пластиковые, хорошая входная железная дверь. Балкон застеклен металлическим остеклением. Квартира освобождена от ...",2900,79068994706 ... Эльвира Ишпахтина,Рио-Люкс,elvira_mag@inbox.ru
1,None,25.0808:04,Однокомнатная,,Центральная 9,1/1,32.00,17.00,7.00,"id:14615. \r\nПродам квартиру с приусадебным участком в д. Ташбулатово ( вблизи озера Банное ).\r\nОтопление газовое, вода- скважина. В квартире сделан косметический ремонт, установлены пластиковые окна, есть подпол .\r\nНа участке – отличная действующая баня, курятник, сарай.\r\nНапротив дом...",1600,79068517952 ... Евгения Гордейчук,Рио-Люкс,gordeychuk.riolux@gmail.com
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,25.0808:04,Многокомнатная,,Горького 12,3/3,149.00,100.00,12.00,"id:11109. \r\nЭКСКЛЮЗИВНЫЙ ВАРИАНТ !\r\n “ Золотой квадрат “ – самое престижное месторасположение в Ленинском районе. \r\nВ продаже шикарная 5-ти комнатная квартира, полностью подготовленная к Вашему проживанию.\r\nКухня-гостиная, кабинет, 3 спальни, 2 санузла, 3 балкона с потрясающим видом.\r\...",12750,79068517952 ... Евгения Гордейчук,Рио-Люкс,gordeychuk.riolux@gmail.com
4,None,25.0808:04,Двухкомнатная,,Советская 155/1,5/5,44.00,29.00,6.00,"id:5056. \r\n!!! Продам 2-комнатную квартиру в хорошем состоянии.\r\nХарактеристики: планировка хрущёвка, этаж 5 из 5, площадь 44 м2, жилая 29 м2, кухня 6 м2, стены панельные, перекрытия железобетонные.\r\nПрекрасная квартира в отличном состоянии, светлая, тёплая, сторона запад. Окна в квартире ...",2930,79514596731 ... Данил Яхин,Рио-Люкс,yahin.riolux@mail.ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,None,20.0811:04,Трехкомнатная нестандартная,Ленинский,Николая Шишка 22,5/5,74.00,56.00,9.00,"Объявление актуально. Продам трёх комнатную квартиру нестандартной планировки по улице Николая Шишка 22.Окна выходят на северо-запад.В квартире сделан дизайнерский ремонт. Окна, двери поменяны. На полу ламинат. Сан узел в кафеле. Балкон застеклен. Остается кухня. Развитая инфраструктура. Красивы...",5500,8-963-478-43-08 ... Ведякина Люцина,Формула недвижимости,8 982 105 20 21
467,None,12.0518:36,Двухкомнатная,,Молодежная 19А,1/5,41.00,22.20,7.20,"ПPОДАЁТCЯ 2 К.КВ., ПО АДРEСУ село Шухободь, ул. Молодежная, 19-А\r\n\r\n- НА 1 ЭТAЖE пятиэтажного дома\r\n\r\n- МETРАЖ: всей квapтиры 41,0 кв. м., жилая 22,2 (комната 12,4 кв. м. комнaтa 9,8 кв. м.) кухня 7,2 кв. м., балкон 3.5 кв. м.\r\n\r\n- СOСТОЯHИЕ: хорошее, заменены окнa, мeжкoмнaтные двер...",930,+79115050320 ... Григорий Голубев,,600-320@mail.ru
468,None,01.0617:01,Двухкомнатная хрущевка,Правобережный,"Им. газеты ""Правда"" 22",5/5,42.50,29.00,6.00,"Продам 2х комнатную ""хрущевку"" в хорошем состоянии, комнаты и сан.узел смежные. Окна, трубы пластиковые, балкон застеклен. Остается кухонный гарнитур. в собственности более 5 лет, один взрослый собственник. Без обременений.",2650,89630958889 ... Макаров Юрий Александрович,""" СИГМА """,
469,None,08.0818:50,Двухкомнатная улучшенная,Орджоникидзевский,Сиреневый проезд 10,3/9,50.00,30.00,9.00,"Продам двухкомнатную улучшенной планировки не торцевую. Очень удобное расположение дома, напротив дома два детских садика, через двор школа, в шаговой доступности сетевые магазины, детская поликлиника, остановки общественного транспорта (маршрутки и трамвай). Окна пластиковые выходят на южную ст...",3350,"8-908-818-12-40, 49-17-15 ... Мещерякова Елена Сергеевна",Новые Технологии,nedv_elen@mail.ru


In [9]:
df.to_csv('df.csv', index=False)